Define CBC Function

In [10]:
# from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
# from cryptography.hazmat.primitives import padding
# from cryptography.hazmat.backends import default_backend
# import os

# def pad(data, block_size):
#     """Pad the data to be a multiple of block_size."""
#     padder = padding.PKCS7(block_size * 8).padder()
#     padded_data = padder.update(data) + padder.finalize()
#     return padded_data

# def cbc_encrypt_16bit(key, plaintext):
#     """Encrypt plaintext using AES in CBC mode and return the first 16 bits of the ciphertext."""
#     # Generate a random IV
#     iv = os.urandom(16)  # 16 bytes for AES

#     # Ensure plaintext length is a multiple of block size (padding)
#     padded_plaintext = pad(plaintext, 16)

#     # Create cipher
#     cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
#     encryptor = cipher.encryptor()

#     # Encrypt plaintext
#     ciphertext = encryptor.update(padded_plaintext) + encryptor.finalize()

#     # Concatenate IV with ciphertext and truncate to 16 bits
#     full_ciphertext = iv + ciphertext
#     truncated_ciphertext = full_ciphertext[:2]  # 16 bits

#     return truncated_ciphertext

# # Example usage
# key = os.urandom(16)  # AES-128 key
# plaintext = b'This is a test message.'

# # Encrypt and truncate
# truncated_ciphertext = cbc_encrypt_16bit(key, plaintext)
# # print(f'Truncated Ciphertext (16 bits): {int.from_bytes(truncated_ciphertext, byteorder="big"):04x}')

# # Convert truncated ciphertext to binary format
# binary_output = ''.join(format(byte, '08b') for byte in truncated_ciphertext)

# print(f'Original plaintext: {plaintext}')
# print(f'Truncated Ciphertext (16 bits): {binary_output}')

In [11]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.backends import default_backend
import os

def pad(data, block_size):
    """Pad the data to be a multiple of block_size."""
    padder = padding.PKCS7(block_size * 8).padder()
    padded_data = padder.update(data) + padder.finalize()
    return padded_data

def bitstring_to_bytes(s):
    """Convert a string of bits to bytes."""
    return int(s, 2).to_bytes((len(s) + 7) // 8, byteorder='big')

def cbc_encrypt_16bit(key, bit_input):
    """Encrypt bit_input using AES in CBC mode and return the first 16 bits of the ciphertext."""

    # Convert key input to bytes
    key_bytes = int(key, 2).to_bytes((len(key) + 7) // 8, byteorder='big')

    # Convert bit input to bytes
    plaintext = bitstring_to_bytes(bit_input)

    # Generate a random IV
    iv = os.urandom(16)  # 16 bytes for AES

    # Ensure plaintext length is a multiple of block size (padding)
    padded_plaintext = pad(plaintext, 16)

    # Create cipher
    cipher = Cipher(algorithms.AES(key_bytes + b'\x00' * (16 - len(key_bytes))), modes.CBC(iv), backend=default_backend())
    encryptor = cipher.encryptor()

    # Encrypt plaintext
    ciphertext = encryptor.update(padded_plaintext) + encryptor.finalize()

    # Concatenate IV with ciphertext and truncate to 16 bits
    full_ciphertext = iv + ciphertext
    truncated_ciphertext = full_ciphertext[:2]  # 16 bits

    return truncated_ciphertext

# Example usage
# key = os.urandom(16)  # AES-128 key
key = '1100110010101010'
bit_input = '110011001010101000'  # Example 16-bit input in bit form

# Encrypt and truncate
truncated_ciphertext = cbc_encrypt_16bit(key, bit_input)

# Convert truncated ciphertext to binary format
binary_output = ''.join(format(byte, '08b') for byte in truncated_ciphertext)

print(f'Original bit input: {bit_input}')
print(f'Truncated Ciphertext (16 bits): {binary_output}')


Original bit input: 110011001010101000
Truncated Ciphertext (16 bits): 1010000010111100


Define Feistel Structure for 16 bits -- Main Hashing Function

In [15]:
def feistel_round(L, R, key):
    """Simple round function for Feistel block."""
    return R, L ^ (R ^ key)  # XOR operation for the round function

def bitstring_to_bytes(s):
    """Convert a string of bits to bytes."""
    return int(s, 2).to_bytes((len(s) + 7) // 8, byteorder='big')

def bytes_to_bitstring(b):
    """Convert bytes to a string of bits."""
    return ''.join(format(byte, '08b') for byte in b)

def feistel_block(bit_input, key, rounds=16):
    """Feistel block encryption with 16-bit input in bitstring form and 16-bit output."""
    # Convert bit input to bytes
    if len(bit_input) != 16:
        raise ValueError("Input bitstring must be exactly 16 bits")

    input_data = bitstring_to_bytes(bit_input)

    # Split input into two 8-bit halves
    L = input_data[0]
    R = input_data[1]

    # Convert key to bytes if it's not already in bytes
    key_bytes = int(key, 2).to_bytes(1, byteorder='big')

    # Perform Feistel rounds
    for i in range(rounds):
        L, R = feistel_round(L, R, key_bytes[0])

    # Combine the halves back into a 16-bit output
    output_data = bytes([L, R])

    # Convert output to bitstring
    output_bitstring = bytes_to_bitstring(output_data)
    return output_bitstring

# Example usage
bit_input = '1100110010101010'  # 16-bit input in bit form
key = '01100110'  # 8-bit key for simplicity

output_bitstring = feistel_block(bit_input, key)
print(f'Input Data (16 bits): {bit_input}')
print(f'Output Data (16 bits): {output_bitstring}')

Input Data (16 bits): 1100110010101010
Output Data (16 bits): 1010101000000000


HMAC Function -- modified

In [13]:
# # Define your HMAC function
# def hmac_custom(message, key, feistel_key):

#   ipad = '00110110' * 2  # 00110110 00110110 in binary
#   opad = '01011100' * 2  # 01011100 01011100 in binary

#   # Convert keys and pads to binary format
#   key_bin = key
#   ipad_bin = ipad
#   opad_bin = opad

#   # XOR operation for inside and outside hash
#   inside_hash_bin = ''.join(str(int(k) ^ int(i)) for k, i in zip(key_bin, ipad_bin))
#   outside_hash_bin = ''.join(str(int(k) ^ int(o)) for k, o in zip(key_bin, opad_bin))

#   """Custom HMAC function using CBC and Feistel."""
#   # Step 1: Concatenate message and inside hash, and pad if necessary

#   concat_message_inside_hash = message + inside_hash_bin
#   if len(concat_message_inside_hash) < 16:
#       concat_message_inside_hash = concat_message_inside_hash.ljust(16, '0')

#   # Step 2: Perform CBC encryption
#   cbc_result = cbc_encrypt_16bit(key, concat_message_inside_hash)

#   #     # Encrypt and truncate
#   # truncated_ciphertext = cbc_encrypt_16bit(key, bit_input)

#   # Convert truncated ciphertext to binary format
#   binary_output = ''.join(format(byte, '08b') for byte in cbc_result)

#   # Convert CBC result to bitstring for Feistel function
#   # cbc_result_bitstring = bytes_to_bitstring(cbc_result)

#   # Step 3: Perform Feistel encryption (16 rounds)
#   feistel_result = feistel_block(binary_output, feistel_key, rounds=16)

#   # Step 4: Concatenate with outside hash, and pad if necessary
#   concat_feistel_outside_hash = feistel_result + outside_hash_bin
#   if len(concat_feistel_outside_hash) < 16:
#       concat_feistel_outside_hash = concat_feistel_outside_hash.ljust(16, '0')

#   # Step 5: Perform CBC encryption again
#   final_cbc_result = cbc_encrypt_16bit(key, concat_feistel_outside_hash)

#   binary_output = ''.join(format(byte, '08b') for byte in final_cbc_result)

#   # # Convert CBC result to bitstring for Feistel function
#   # final_cbc_result_bitstring = bytes_to_bitstring(final_cbc_result)

#   # Step 6: Perform Feistel encryption again (16 rounds)
#   final_feistel_result = feistel_block(binary_output, feistel_key, rounds=16)

#   return final_feistel_result

# # Example usage
# message = '1100110010101010'  # Example 16-bit message
# feistel_key = '01100110'  # 8-bit key for Feistel rounds
# key = os.urandom(16)

# hmac_result = hmac_custom(message, key, feistel_key)
# print(f'HMAC Result (16 bits): {hmac_result}')

In [17]:
# Define your HMAC function
def hmac_custom(message, key, feistel_key):
    ipad = '00110110' * 2  # 00110110 00110110 in binary
    opad = '01011100' * 2  # 01011100 01011100 in binary

    # Convert keys and pads to binary format
    # key_bin = bytes_to_bitstring(key)
    ipad_bin = ipad
    opad_bin = opad

    # XOR operation for inside and outside hash
    inside_hash_bin = ''.join(str(int(k) ^ int(i)) for k, i in zip(key, ipad_bin))
    outside_hash_bin = ''.join(str(int(k) ^ int(o)) for k, o in zip(key, opad_bin))

    """Custom HMAC function using CBC and Feistel."""
    # Step 1: Concatenate message and inside hash, and pad if necessary
    concat_message_inside_hash = message + inside_hash_bin
    if len(concat_message_inside_hash) < 16:
        concat_message_inside_hash = concat_message_inside_hash.ljust(16, '0')

    # Step 2: Perform CBC encryption
    # cbc_result = cbc_encrypt_16bit(key, bitstring_to_bytes(concat_message_inside_hash))
    cbc_result = cbc_encrypt_16bit(key, concat_message_inside_hash)

    # Convert truncated ciphertext to binary format
    binary_output = ''.join(format(byte, '08b') for byte in cbc_result)

    # Step 3: Perform Feistel encryption (16 rounds)
    feistel_result = feistel_block(binary_output, feistel_key, rounds=16)

    # Step 4: Concatenate with outside hash, and pad if necessary
    concat_feistel_outside_hash = feistel_result + outside_hash_bin
    if len(concat_feistel_outside_hash) < 16:
        concat_feistel_outside_hash = concat_feistel_outside_hash.ljust(16, '0')

    # Step 5: Perform CBC encryption again
    final_cbc_result = cbc_encrypt_16bit(key, concat_feistel_outside_hash)

    binary_output = ''.join(format(byte, '08b') for byte in final_cbc_result)

    # Step 6: Perform Feistel encryption again (16 rounds)
    final_feistel_result = feistel_block(binary_output, feistel_key, rounds=16)

    return final_feistel_result

# # Example usage
# message = '1100110010101010'  # Example 16-bit message
# feistel_key = '01100110'  # 8-bit key for Feistel rounds
# # key = os.urandom(16)
# key = '1100110010101010'

# hmac_result = hmac_custom(message, key, feistel_key)
# print(f'HMAC Result (16 bits): {hmac_result}')

def main():
    message = input("Enter the message (16 bits): ")
    key = input("Enter the key (16 bits): ")
    feistel_key = input("Enter feistel key (16 bits): ")

    hmac_result = hmac_custom(message, key, feistel_key)
    print()
    print("HMAC RESULT:", hmac_result)

if __name__ == "__main__":
    main()

Enter the message (16 bits): 1100110010101010
Enter the key (16 bits): 1100110010101010
Enter feistel key (16 bits): 01100110

HMAC RESULT: 0001011000100010
